In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()



In [2]:


# One-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)




In [3]:
# 2. Data Augmentation + Resize on the fly
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow from numpy arrays with resizing inside the generator
train_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=64,
    shuffle=True
)

test_generator = test_datagen.flow(
    x_test, y_test,
    batch_size=64,
    shuffle=False
)

# Override generator to resize dynamically
def resize_generator(gen):
    for batch_x, batch_y in gen:
        batch_x_resized = tf.image.resize(batch_x, (224,224))  # resize per batch
        yield batch_x_resized, batch_y

train_generator_resized = resize_generator(train_generator)
test_generator_resized = resize_generator(test_generator)

In [4]:
# 3. Load Pre-trained ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze all layers first
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze last 50 layers
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [5]:
# 4. Add custom classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])



In [6]:
# 5. Compile model (smaller learning rate for fine-tuning)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator_resized,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=test_generator_resized,
    validation_steps=len(test_generator),
    callbacks=[
        EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
        ModelCheckpoint('best_resnet50_cifar10.h5', monitor='val_accuracy', save_best_only=True)
    ]
)

Epoch 1/20


In [ ]:
# 8. Evaluate
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("Test Accuracy with Fine-tuned ResNet50:", test_acc)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



In [2]:
# Enable mixed precision (optional, only if GPU supports it)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")





The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [3]:
# 1. Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)


In [4]:

# 2. Data pipeline function
def preprocess_image(image, label):
    image = tf.image.resize(image, (224,224))         # resize for ResNet50
    image = preprocess_input(image)                   # match ResNet50 preprocessing
    return image, label

def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label



In [5]:
# 3. Build tf.data pipelines
batch_size = 8

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(5000) \
                   .map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE) \
                   .map(augment_image, num_parallel_calls=tf.data.AUTOTUNE) \
                   .batch(batch_size) \
                   .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE) \
                 .batch(batch_size) \
                 .prefetch(tf.data.AUTOTUNE)



In [6]:
# 4. Load ResNet50
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))

# Freeze then unfreeze last 50 layers
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-50:]:
    layer.trainable = True



In [7]:
# 5. Custom classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax', dtype="float32")  # force float32 output (for mixed precision)
])



In [8]:
# 6. Compile
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [9]:
# 7. Callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_resnet50_cifar10.h5', monitor='val_accuracy', save_best_only=True)
]


In [10]:
# 8. Train
history = model.fit(
    train_ds,
    epochs=20,
    validation_data=test_ds,
    callbacks=callbacks
)


Epoch 1/20


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/sequential/resnet50/conv4_block6_2_bn/FusedBatchNormGradV3' defined at (most recent call last):
    File "C:\Users\manis\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\manis\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\manis\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\manis\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\manis\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\manis\AppData\Local\Programs\Pyt

In [ ]:
# 9. Evaluate
test_loss, test_acc = model.evaluate(test_ds)
print("Final Test Accuracy:", test_acc)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import mixed_precision

# Enable mixed precision if GPU supports it
mixed_precision.set_global_policy('mixed_float16')

# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Preprocess and augment functions with enhanced augmentations
def preprocess_image(image, label):
    image = tf.image.resize(image, (128, 128))
    image = preprocess_input(image)
    return image, label

def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_saturation(image, 0.6, 1.4)
    image = tf.image.random_hue(image, 0.1)
    return image, label

# Dataset pipeline with reduced batch size and limited parallelism
batch_size = 4

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(2000) \
                   .map(preprocess_image, num_parallel_calls=1) \
                   .map(augment_image, num_parallel_calls=1) \
                   .batch(batch_size) \
                   .prefetch(1)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(preprocess_image, num_parallel_calls=1) \
                 .batch(batch_size) \
                 .prefetch(1)

# Load ResNet50 base model, unfreeze last 20 layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
for layer in base_model.layers[:-20]:
    layer.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Build classification head with batch normalization
inputs = tf.keras.Input(shape=(128, 128, 3))
x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax', dtype='float32')(x)  # Force output to float32

model = tf.keras.Model(inputs, outputs)

# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks: Early stopping and LR reduction on plateau
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

# Train the model
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    callbacks=callbacks
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_ds)
print(f"Final Test Accuracy: {test_acc:.4f}")
